In [1]:
import json
import requests
import sys
import warnings
from datetime import datetime, time, timedelta
from selenium import webdriver
#from seleniumwire import webdriver

GECKO_PATH = 'C:\\Users\\25052288840\\Downloads\\chromedriver.exe'
sys.path.insert(0, '../../../ajna_docs/commons/')
sys.path.insert(0, '../../../commons/')
sys.path.insert(0, '../../../virasana/')
from virasana.integracao.due import raspa_due

warnings.simplefilter("ignore")

VIRASANA_URL = "https://ajna.labin.rf08.srf/virasana/"

C:\Users\25052288840\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-12-08 14:50:30,732 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\access.log


In [2]:
def do_update(driver, tipo_manifesto):
    if tipo_manifesto == 'VAZIO':
            conteineres_ids = raspa_due.raspa_containers_vazios_sem_due(
            datainicial, datafinal)
    else:
        conteineres_ids = raspa_due.raspa_containers_sem_due(
            datainicial, datafinal, tipo_manifesto)
    print('%s contêineres a consultar em POS ACD' % len(conteineres_ids))
    conteineres_listadue = raspa_due.get_dues_pos_acd(driver, list(conteineres_ids.keys()))
    setdues = set()
    for conteiner, dues in conteineres_listadue.items():
        for due in dues:
            setdues.add(due)
    print('%s contêineres a consultar em DUE DETALHE' % len(setdues))
    due_detalhe = raspa_due.detalha_dues(driver, conteineres_listadue)
    print(len(due_detalhe))
    pacote_carregamento = {}
    for conteiner, numeros_dues in conteineres_listadue.items():
        _id = conteineres_ids[conteiner]
        lista_dues = []
        for due in numeros_dues:
            if due is not None:
                pacote = raspa_due.monta_due_ajna(due_detalhe[due])
                lista_dues.append({'numero': due, **pacote})
        if numeros_dues and len(numeros_dues) > 0:
            pacote_carregamento[_id] = lista_dues
    print('Inserindo %d dues...' % len(pacote_carregamento))
    r = requests.post(VIRASANA_URL + "dues/update", json=pacote_carregamento, verify=False)
    print(r.status_code)
    print(r.text)

In [3]:
diaapesquisar = datetime.today() - timedelta(days=0)
datainicial = datetime.strftime(datetime.combine(diaapesquisar, time.min), '%Y-%m-%d  %H:%M:%S')
datafinal = datetime.strftime(datetime.combine(diaapesquisar, time.max), '%Y-%m-%d %H:%M:%S')
print(datainicial, datafinal)

2022-12-08  00:00:00 2022-12-08 23:59:59


In [4]:
#driver.close()


In [5]:
options = webdriver.ChromeOptions()
#options.add_argument('--no-sandbox')
#options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(GECKO_PATH, options = options)
# driver = webdriver.Chrome()


driver.get("https://www.suiterfb.receita.fazenda")
driver.get("https://portalunico.siscomex.gov.br")

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 105
Current browser version is 107.0.5304.88 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	Ordinal0 [0x0069DF13+2219795]
	Ordinal0 [0x00632841+1779777]
	Ordinal0 [0x0054423D+803389]
	Ordinal0 [0x005664AC+943276]
	Ordinal0 [0x005619F0+924144]
	Ordinal0 [0x0055F179+913785]
	Ordinal0 [0x005936B9+1128121]
	Ordinal0 [0x0059331A+1127194]
	Ordinal0 [0x0058E616+1107478]
	Ordinal0 [0x00567F89+950153]
	Ordinal0 [0x00568F56+954198]
	GetHandleVerifier [0x00992CB2+3040210]
	GetHandleVerifier [0x00982BB4+2974420]
	GetHandleVerifier [0x00736A0A+565546]
	GetHandleVerifier [0x00735680+560544]
	Ordinal0 [0x00639A5C+1808988]
	Ordinal0 [0x0063E3A8+1827752]
	Ordinal0 [0x0063E495+1827989]
	Ordinal0 [0x006480A4+1867940]
	BaseThreadInitThunk [0x75EEFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77837B5E+286]
	RtlGetAppContainerNamedObjectPat

In [ ]:
js = '''var xhr = new XMLHttpRequest();
xhr.open('POST', 'https://portalunico.siscomex.gov.br/portal/proxy/authenticate', false);
xhr.setRequestHeader('Credential-Type', 'USUARIO_PORTAL');
xhr.setRequestHeader('Role-Type', 'rfb');
xhr.send('');
return xhr.response;'''
result = driver.execute_script(js);
token = json.loads(result)['token']

In [ ]:
from html import escape
print(token)
print(escape(token))

In [ ]:
driver.get('https://portalunico.siscomex.gov.br/portal/#/')
js = '''var xhr = new XMLHttpRequest();
xhr.open('GET', 'https://portalunico.siscomex.gov.br/cct/#/consulta-estoque-pos-acd', false);
xhr.setRequestHeader('Credential-Type', 'USUARIO_PORTAL');
xhr.setRequestHeader('Role-Type', 'rfb');
xhr.setRequestHeader('X-CSRF-Token', '%s');
xhr.send('');
return xhr.response;''' % escape(token)
print(js)
result = driver.execute_script(js);


In [ ]:
import requests
requests.get('https://portalunico.siscomex.gov.br/cct/api/deposito-carga/consultar-estoque-pos-acd?numeroConteiner')

In [ ]:
print(result)

In [ ]:
headers = {}
for request in driver.requests:
    headers = {*headers, *request.headers}
print(headers)

In [ ]:
def interceptor(request):
    request.headers['Credential-Type'] = 'USUARIO_PORTAL'
    request.headers['Role-Type'] = 'rfb'
    request.method = 'POST'
driver.request_interceptor = interceptor
driver.get("https://portalunico.siscomex.gov.br/portal/proxy/authenticate")



In [ ]:
break!!!

In [ ]:
driver = webdriver.Chrome(GECKO_PATH)
try:
    portal_url = 'https://portalunico.siscomex.gov.br/portal/'
    driver.get(portal_url)
    div_governo = driver.find_element_by_id('grp_governo')
    div_governo.click()
    li_rfb = driver.find_element_by_id("prf_rfb")
    li_rfb.click()
    btn_certificado = driver.find_element_by_id("btn_certificado")
    btn_certificado.click()
finally:
    pass



In [ ]:
driver.close

In [ ]:
driver.requests[-1].headers


In [ ]:
driver.requests[-1].headers

In [ ]:
for request in driver.requests:
    if request.response:
        print(
            request.url,
            request.response.status_code,
            request.response.headers
        )

In [ ]:
headers

In [ ]:

"https://portalunico.siscomex.gov.br/portal/proxy/authenticate"

In [ ]:
#driver = webdriver.Chrome(GECKO_PATH)
try:
    #raspa_due.auth_suite_rfb(driver)
    do_update(driver, 'lce')
    do_update(driver, None)
    do_update(driver, 'VAZIO')
finally:
    pass
    # driver.close()


In [ ]:
params = {'query':
               {'metadata.carga.conhecimento.conhecimento': '152005011610119',
                 'metadata.contentType': 'image/jpeg',
                'metadata.numeroinformado': 'PONU7791526'
                 },
          'projection':
               {'_id': 1}
         }
r = requests.post('https://ajna.labin.rf08.srf/virasana/grid_data', json=params, verify=False)


In [ ]:
r.content

In [ ]:
lista = r.json()

In [ ]:
_id = lista[0]['_id']

In [ ]:
_id

In [ ]:
params = {'query':
               {'metadata.dataescaneamento': {'$gt': '2020-12-09 00:00:00'},
                 'metadata.contentType': 'image/jpeg'},
          'projection':
               {'_id': 1}
         }
r = requests.post('https://ajna.labin.rf08.srf/virasana/grid_data', json=params, verify=False)
r.content

In [ ]:
import json 
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--allow-running-insecure-content')
options.add_argument('--ignore-certificate-errors')

driver = webdriver.Chrome(GECKO_PATH, chrome_options=options)
try:
    r = driver.get('https://ajna.labin.rf08.srf/ajnaapi/api/login_certificado')
    token = json.loads(raspa_due.limpa_pre(driver.page_source))['access_token']
    print(token)
finally:
    driver.close()

In [ ]:
headers = {'Authorization': f'Bearer {token}'}
params = {'query':
               {'metadata.numeroinformado': 'MSCU0109528',
                 'metadata.contentType': 'image/jpeg'},
          'projection':
               {'_id': 1}
         }
r = requests.post('https://ajna.labin.rf08.srf/ajnaapi/api/grid_data', json=params, headers=headers, verify=False)
r.content

In [ ]:
_id = r.json()[0]['_id']
r = requests.get(f'https://ajna.labin.rf08.srf/ajnaapi/api/image/{_id}', headers=headers, verify=False)
r.content